##Wellness 심리 상담 데이터 + ChatBot 데이터를 활용한 KOGPT2 학습##

1. 구글 드라이브 연동

* 모델 파일, 훈련 데이터가 저장 된 구글 드라이브 디렉토리와 Colab을 연동시킨다

* 런타임 유형을 GPU로 변경시킨다.(학습 시간이 꽤 걸림)

In [1]:
#GPU 연동 확인
!nvidia-smi

Thu Oct 29 04:57:54 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls drive/'My Drive'/'RogerHeederer'/'ChatBot'/'KoGPT2_Wellness'

checkpoint  dataloader	model_play  requirements.txt
data	    model	preprocess  train


필요한 패키지들 설치

In [4]:
!pip install -r drive/'My Drive'/'RogerHeederer'/'ChatBot'/'KoGPT2_Wellness'/requirements.txt

     |████████████████████████████████| 778kB 10.4MB/s 
     |████████████████████████████████| 2.9MB 32.8MB/s 
     |████████████████████████████████| 1.1MB 53.6MB/s 
     |████████████████████████████████| 890kB 46.0MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
  Created wheel for kss: filename=kss-1.3.1-cp36-cp36m-linux_x86_64.whl size=251557 sha256=f0212e2dca388c50deda8c29ba27415fe9e3542fc0b0da49235f43f4320c2d58
  Stored in directory: /root/.cache/pip/wheels/8b/98/d1/53f75f89925cd95779824778725ee3fa36e7aa55ed26ad54a8
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=612e8eccb4f52c17e266bb6d0ebbfe77e6f2e3f30bd0d5eb6cc4138fadcb6bc1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built kss sacremoses
ERROR: transformers 3.0.2 has requirement tokenizers==0.8.1.rc1, but you'll have tokenizers 0.9.3 which is incompatible.


트랜스포머의 토크나이져 버젼이 맞지 않는다고 하여, 기 설치된 토크나이저를 제거 한 후 새로운 토크나이져 설치

In [6]:
!pip uninstall tokenizers

In [7]:
!pip install tokenizers==0.8.1.rc1

     |████████████████████████████████| 3.0MB 5.9MB/s 


**자 이제 본격적으로 KoGPT2를 정신상담 데이터 셋으로 훈련시켜보자**

In [8]:
# 훈련 데이터, 데이터 로더 등을 불러오기 위해 경로 설정
import sys
sys.path.append('drive/My Drive/RogerHeederer/ChatBot')

In [9]:
import os
import numpy as np
from tqdm import tqdm #파이썬 진행표시바 라이브러리

import torch
from torch.utils.data import dataloader

#--내가 구성한 폴더 속의 데이터 로더를 불러오는 것임--#
#--원래 코드에는 DialogLM 이라는 폴더로 구성되어 있었는데--#
#--초급자 입장에서는 공식 라이브러리로 간주하여, import가 안되서 꽤 헤맸던 요소 --#

#KoGPT2_Wellness폴더의 dataloader폴더의 total_data_load.py에서 import 클래스 이름
from KoGPT2_Wellness.dataloader.total_data_load import TotalAutoRegressiveDataset
#KoGPT2_Wellness폴더의 model 폴더의 kogpt2.py에서 import 클래스 이름
from KoGPT2_Wellness.model.kogpt2 import DialogKoGPT2

In [10]:
torch.cuda.is_available()

True

##KoGPT2 Training by Total dataset(Wellness+chatbot)##

GPT-2 is auto-regressive but Bert is not
 - After each token is produced, that token is added to the sequence of inputs

**AutoRegressive 구조**

각각의 토큰이 생성이 되면, 그 생성된 토큰이 그 다음 토큰을 생성시키기 위한 인풋으로 사용되는 구조

In [11]:
root_path = "drive/My Drive/RogerHeederer/ChatBot/KoGPT2_Wellness"
data_path = f"{root_path}/data/total_train.txt"
checkpoint_path = f"{root_path}/checkpoint"
save_ckpt_path = f"{checkpoint_path}/total_autoregressive.pth"

In [17]:
n_epoch = 6    # Num of Epoch/ 기존 오리지널 코드는 epoch 5 늘려봄
batch_size = 2 # 배치 사이즈
ctx = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(ctx)
save_step = 100 # 학습 저장 주기
learninig_rate = 5e-5 # Learning Rate

In [15]:
dataset = TotalAutoRegressiveDataset(data_path) # total_train.txt를 데이터 셋으로 구성 후 로딩
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [13]:
model = DialogKoGPT2() #모델 이니셜라이즈
model.to(device)

DialogKoGPT2(
  (kogpt2): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50000, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0): Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (1): Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
         

In [18]:
loss_fct = torch.nn.CrossEntropyLoss(ignore_index=3)
optimizer = torch.optim.Adam(model.parameters(), lr=learninig_rate)

losses =[]
for epoch in range(n_epoch):
    count = 0
    with tqdm(total=len(train_loader), desc=f"Train({epoch})") as pbar:
        for i, data in enumerate(train_loader):
            optimizer.zero_grad()
            data = torch.stack(data)  # list of Tensor로 구성되어 있기 때문에 list를 stack을 통해 변환해준다.
            data = data.transpose(1, 0)
            data= data.to(ctx)

            outputs = model(data, labels=data)
            _, logits = outputs[:2]

            # Shift so that tokens < n predict n
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = data[..., 1:].contiguous()

            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            loss.backward()
            optimizer.step()

            losses.append(loss.item())

            # if count % 10 == 0:
            #     print('epoch no.{} train no.{}  loss = {}'.format(epoch, count + 1, loss))
            if (count > 0 and count % save_step == 0) or (len(data) < batch_size):
                torch.save({
                    'epoch': epoch,
                    'train_no': count,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss}, save_ckpt_path)
            
            count += 1
            pbar.update(1)
            pbar.set_postfix_str(f"Loss: {loss.item():.3f} ({np.mean(losses):.3f})")

Train(5): 100%|██████████| 12531/12531 [1:07:37<00:00,  3.09it/s, Loss: 0.621 (1.209)]
